# Experimentation Only

In [1]:
import sys
!{sys.executable} -m pip install tqdm

You are using pip version 9.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [32]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from datetime import datetime
import datetime as dt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

#For Prediction
from sklearn import metrics
from sklearn import preprocessing, cross_validation

import keras
from keras.layers import Dense, Dropout, Input
from keras.layers import Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.models import Model,Sequential
from keras.datasets import mnist
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, RMSprop
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization


In [33]:
# Read Datasets 
research_data_raw = pd.read_csv('Datasets/bitcoin_research_data_v2.csv')
# Drop unnecessary columns
research_data = research_data_raw.drop('Unnamed: 0', axis=1)
research_data.head()

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD,Popularity,Popularity_Normalized
0,2019-05-11,BTCUSD,6337.9,6984.8,6337.9,6793.5,9855.87,66133073.75,12.0,1200.0
1,2019-05-10,BTCUSD,6151.4,6430.0,6110.1,6337.9,9872.86,62166515.28,12.0,1200.0
2,2019-05-09,BTCUSD,5939.6,6174.0,5933.2,6151.4,7405.18,44816872.39,12.0,1200.0
3,2019-05-08,BTCUSD,5744.0,5983.2,5660.0,5939.6,6007.11,35184697.52,12.0,1200.0
4,2019-05-07,BTCUSD,5687.4,5955.8,5687.4,5906.7,3668.12,21496575.65,12.0,1200.0


In [34]:
# Convert data to milliseconds
research_data['Epoch'] = pd.to_datetime(research_data['Date'] + ' 23:59:59.000')
research_data['Epoch'] = (research_data['Epoch'] - dt.datetime(1970,1,1)).dt.total_seconds().astype('int64')
research_data.head()

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD,Popularity,Popularity_Normalized,Epoch
0,2019-05-11,BTCUSD,6337.9,6984.8,6337.9,6793.5,9855.87,66133073.75,12.0,1200.0,1557619199
1,2019-05-10,BTCUSD,6151.4,6430.0,6110.1,6337.9,9872.86,62166515.28,12.0,1200.0,1557532799
2,2019-05-09,BTCUSD,5939.6,6174.0,5933.2,6151.4,7405.18,44816872.39,12.0,1200.0,1557446399
3,2019-05-08,BTCUSD,5744.0,5983.2,5660.0,5939.6,6007.11,35184697.52,12.0,1200.0,1557359999
4,2019-05-07,BTCUSD,5687.4,5955.8,5687.4,5906.7,3668.12,21496575.65,12.0,1200.0,1557273599


In [35]:
research_data_imp2 = research_data.copy()
research_data_imp2.sort_values(by='Date', inplace=True, ascending=True) # Sort by date ascending
forecast_out = int(math.ceil(0.0001 * len(research_data_imp2))) # 0.01% of data
#forecast_out = int(math.ceil(0.0025 * len(research_data_imp2))) # 0.05% of data
print(len(research_data_imp2))
print(forecast_out)

1808
1


In [36]:
research_data_imp2['Prediction'] = research_data_imp2['Close'].shift(-forecast_out)
research_data_imp2.tail()

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD,Popularity,Popularity_Normalized,Epoch,Prediction
4,2019-05-07,BTCUSD,5687.4,5955.8,5687.4,5906.7,3668.12,21496575.65,12.0,1200.0,1557273599,5939.6
3,2019-05-08,BTCUSD,5744.0,5983.2,5660.0,5939.6,6007.11,35184697.52,12.0,1200.0,1557359999,6151.4
2,2019-05-09,BTCUSD,5939.6,6174.0,5933.2,6151.4,7405.18,44816872.39,12.0,1200.0,1557446399,6337.9
1,2019-05-10,BTCUSD,6151.4,6430.0,6110.1,6337.9,9872.86,62166515.28,12.0,1200.0,1557532799,6793.5
0,2019-05-11,BTCUSD,6337.9,6984.8,6337.9,6793.5,9855.87,66133073.75,12.0,1200.0,1557619199,NaN


In [37]:
def load_data():
    X = np.array(research_data_imp2.drop(['Prediction', 'Symbol', 'Date', 'Popularity'], 1)) # Create X with numeric columns only
    # X = np.array(research_data_imp2['Close'])
    # X = research_data_imp2['Close'].to_frame()
    X = preprocessing.scale(X)
    X_forecast = X[-forecast_out:]
    X = X[:-forecast_out]

    # research_data_imp2.dropna(inplace=True)

    y = np.array(research_data_imp2['Prediction'])
    y = y[:-forecast_out]

    X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)
    
    return [X_train, X_test, y_train, y_test]

In [38]:
X_train, X_test, y_train, y_test = load_data()
print(X_train.shape)
print(y_train.shape)

(1445, 8)
(1445,)


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [187]:
def discriminator_net():
    D = Sequential()
    depth = 64
    dropout = 0.4
    # In: 28 x 28 x 1, depth = 1
    # Out: 14 x 14 x 1, depth=64
    
    #input_shape = (1445, 8, 1)
    input_shape = (1445, 8, 1)
    D.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape, padding='same'))
    D.add(LeakyReLU(alpha=0.2))
    D.add(Dropout(dropout))

    D.add(Conv2D(depth*2, 5, strides=2, padding='same'))
    D.add(LeakyReLU(alpha=0.2))
    D.add(Dropout(dropout))

    D.add(Conv2D(depth*4, 5, strides=2, padding='same'))
    D.add(LeakyReLU(alpha=0.2))
    D.add(Dropout(dropout))

    D.add(Conv2D(depth*8, 5, strides=1, padding='same'))
    D.add(LeakyReLU(alpha=0.2))
    D.add(Dropout(dropout))

    # Out: 1-dim probability
    D.add(Flatten())
    D.add(Dense(1))
    D.add(Activation('sigmoid'))
    D.summary()
    return D

In [188]:
def generator_net():
    G = Sequential()
    dropout = 0.4
    depth = 64+64+64+64
    dim = 2
    # In: 100
    # Out: dim x dim x depth
    
    G.add(Dense(dim*dim*depth, input_dim=100))
    G.add(BatchNormalization(momentum=0.9))
    G.add(Activation('relu'))
    G.add(Reshape((dim, dim, depth)))
    G.add(Dropout(dropout))

    # In: dim x dim x depth
    # Out: 2*dim x 2*dim x depth/2
    G.add(UpSampling2D())
    G.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
    G.add(BatchNormalization(momentum=0.9))
    G.add(Activation('relu'))

    G.add(UpSampling2D())
    G.add(Conv2DTranspose(int(depth/4), 5, padding='same'))
    G.add(BatchNormalization(momentum=0.9))
    G.add(Activation('relu'))

    G.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
    G.add(BatchNormalization(momentum=0.9))
    G.add(Activation('relu'))

    # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
    G.add(Conv2DTranspose(1, 5, padding='same'))
    G.add(Activation('sigmoid'))
    G.summary()
    return G

In [189]:
def discriminator_model():
    optimizer = RMSprop(lr=0.0002, decay=6e-8)
    DM = Sequential()
    DM.add(discriminator_net())
    DM.compile(loss='binary_crossentropy', optimizer=optimizer,\
        metrics=['accuracy'])
    return DM

In [190]:
def adversarial_model():
    optimizer = RMSprop(lr=0.0001, decay=3e-8)
    AM = Sequential()
    AM.add(generator_net())
    AM.add(discriminator_net())
    AM.compile(loss='binary_crossentropy', optimizer=optimizer,\
        metrics=['accuracy'])


In [191]:
discriminator =  discriminator_model()
adversarial = adversarial_model()
generator = generator_net()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_62 (Conv2D)           (None, 723, 4, 64)        1664      
_________________________________________________________________
leaky_re_lu_49 (LeakyReLU)   (None, 723, 4, 64)        0         
_________________________________________________________________
dropout_67 (Dropout)         (None, 723, 4, 64)        0         
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 362, 2, 128)       204928    
_________________________________________________________________
leaky_re_lu_50 (LeakyReLU)   (None, 362, 2, 128)       0         
_________________________________________________________________
dropout_68 (Dropout)         (None, 362, 2, 128)       0         
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 181, 1, 256)       819456    
__________

In [125]:
def train(train_steps=2000, batch_size=256, save_interval=0):
    noise_input = None
    if save_interval>0:
        noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
    for e in range(train_steps):
        print("Epoch %d" %e)
        for i in tqdm(range(batch_size)):
            #images_train = X_train[np.random.randint(0, X_train.shape[0], size=batch_size), :, :, :]
            images_train = X_train[np.random.randint(0,X_train.shape[0],size=batch_size)]
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            images_fake = generator.predict(noise)
            
            print(images_train.shape)
            print(images_fake.shape)
            x = np.concatenate((images_train, images_fake))
            y = np.ones([2*batch_size, 1])
            y[batch_size:, :] = 0
            d_loss = discriminator.train_on_batch(x, y)

            y = np.ones([batch_size, 1])
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            a_loss = adversarial.train_on_batch(noise, y)
            log_mesg = "%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
            log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
            print(log_mesg)
            #if save_interval>0:
            #    if (i+1)%save_interval==0:
            #        plot_images(save2file=True, samples=noise_input.shape[0],noise=noise_input, step=(i+1))

In [126]:
#train(train_steps=10000, batch_size=256, save_interval=500)
train(train_steps=1, batch_size=256, save_interval=0)

  0%|          | 0/256 [00:00<?, ?it/s]

Epoch 0
(256, 8)
(256, 8, 8, 1)


ValueError: all the input arrays must have same number of dimensions